In [1]:
from builtins import range
import numpy as np

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def eval_numerical_gradient_array(f, x, df, h=1e-5):
    """
    Evaluate a numeric gradient for a function that accepts a numpy
    array and returns a numpy array.
    """
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index

        oldval = x[ix]
        x[ix] = oldval + h
        pos = f(x).copy()
        x[ix] = oldval - h
        neg = f(x).copy()
        x[ix] = oldval
        
        #print('---')
        #print(pos.shape)
        #print(neg.shape)
        #print(grad.shape)
        #print('-----')
        grad[ix] = np.sum((pos - neg) * df) / (2 * h)
        it.iternext()
    return grad

def get_im2col_indices(x_shape, field_height, field_width, padding=1, stride=1):
    # First figure out what the size of the output should be
    N, C, H, W = x_shape
    assert (H + 2 * padding - field_height) % stride == 0
    assert (W + 2 * padding - field_height) % stride == 0
    out_height = (H + 2 * padding - field_height) // stride + 1
    out_width = (W + 2 * padding - field_width) // stride + 1

    i0 = np.repeat(np.arange(field_height), field_width)
    i0 = np.tile(i0, C)
    i1 = stride * np.repeat(np.arange(out_height), out_width)
    j0 = np.tile(np.arange(field_width), field_height * C)
    j1 = stride * np.tile(np.arange(out_width), out_height)
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)

    k = np.repeat(np.arange(C), field_height * field_width).reshape(-1, 1)

    return (k, i, j)


def im2col_indices(x, field_height, field_width, padding=1, stride=1):
    """ An implementation of im2col based on some fancy indexing """
    # Zero-pad the input
    p = padding
    x_padded = np.pad(x, ((0, 0), (0, 0), (p, p), (p, p)), mode='constant')

    k, i, j = get_im2col_indices(x.shape, field_height, field_width, padding,
                                 stride)

    cols = x_padded[:, k, i, j]
    C = x.shape[1]
    cols = cols.transpose(1, 2, 0).reshape(field_height * field_width * C, -1)
    return cols


def col2im_indices(cols, x_shape, field_height=3, field_width=3, padding=1,
                   stride=1):
    """ An implementation of col2im based on fancy indexing and np.add.at """
    N, C, H, W = x_shape
    H_padded, W_padded = H + 2 * padding, W + 2 * padding
    x_padded = np.zeros((N, C, H_padded, W_padded), dtype=cols.dtype)
    k, i, j = get_im2col_indices(x_shape, field_height, field_width, padding,
                                 stride)
    cols_reshaped = cols.reshape(C * field_height * field_width, -1, N)
    cols_reshaped = cols_reshaped.transpose(2, 0, 1)
    np.add.at(x_padded, (slice(None), k, i, j), cols_reshaped)
    if padding == 0:
        return x_padded
    return x_padded[:, :, padding:-padding, padding:-padding]

def zero_pad(X, pad):
    X_pad = np.pad(X, ((0, 0), (0,0), (pad, pad), (pad, pad)), 'constant', constant_values=(0, 0))
    return X_pad

def im2col_sliding(image, filter_height=3, filter_width=3, 
                   padding=0, stride=1):
    
    M, C, h, w, = image.shape
    x_padded = np.pad(x, ((0, 0), (0, 0), (padding, padding), (padding, padding)), 
                      mode='constant')    
    h_new = int((h - filter_height + 2*padding) / stride + 1)
    w_new = int((w - filter_width + 2*padding) / stride + 1)
    
    output_vectors = np.zeros((filter_width*filter_height*C, M*h_new*w_new), dtype=image.dtype)
    
    itr = 0
    for i in range(h_new):
        for j in range(w_new):
             for m in range(M):
                    start_i = stride * i
                    end_i = stride * i + filter_width
                    start_j = stride * j
                    end_j = stride * j + filter_height
                    output_vectors[:, itr] = x_padded[m, :, start_i:end_i, start_j:end_j].ravel()
                    itr += 1                    
    return output_vectors

def col2img_sliding(cols,  x_shape, filter_height=3, filter_width=3, 
                    padding=0, stride=1):
    N, C, H, W = x_shape
    H_padded, W_padded = H + 2 * padding, W + 2 * padding
    x_padded = np.zeros((N, C, H_padded, W_padded), dtype=cols.dtype)
    
    idx = 0
    for i in range(0, H_padded - filter_height + 1, stride):
        for j in range(0, W_padded - filter_width + 1, stride):
            for m in range(N):
                col = cols[:, idx]
                col = col.reshape((C, filter_height, filter_width))            
                x_padded[m, :, i:i+filter_height, j:j+filter_width] += col  
                idx += 1
    if padding > 0:
        return x_padded[:, :, padding:-padding, padding:-padding]
    else:
        return x_padded

In [2]:
x = np.arange(16).reshape((1, 1, 4, 4))
print(x)
print()

i2cs = im2col_indices(x, 2, 2, 0, 2)
c2ii = col2im_indices(i2cs, x.shape, 2, 2, 0, 2)
c2is = col2img_sliding(i2cs, x.shape, 2, 2, 0, 2)

print(i2cs)
print()
print(c2ii)
print()
print(c2is)

print()
print(rel_error(c2ii, c2is))

[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]
   [12 13 14 15]]]]

[[ 0  2  8 10]
 [ 1  3  9 11]
 [ 4  6 12 14]
 [ 5  7 13 15]]

[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]
   [12 13 14 15]]]]

[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]
   [12 13 14 15]]]]

0.0


In [3]:
def conv_forward(X, W, b, stride=1, padding=1):
    cache = W, b, stride, padding
    n_filters, d_filter, h_filter, w_filter = W.shape
    n_x, d_x, h_x, w_x = X.shape
    h_out = (h_x - h_filter + 2 * padding) / stride + 1
    w_out = (w_x - w_filter + 2 * padding) / stride + 1

    if not h_out.is_integer() or not w_out.is_integer():
        raise Exception('Invalid output dimension!')

    h_out, w_out = int(h_out), int(w_out)

    X_col = im2col_sliding(X, h_filter, w_filter, padding=padding, stride=stride)
    W_col = W.reshape(n_filters, -1)

    out = W_col @ X_col + b
    out = out.reshape(n_filters, h_out, w_out, n_x)
    out = out.transpose(3, 0, 1, 2)

    cache = (X, W, b, stride, padding, X_col)

    return out, cache


def conv_backward(dout, cache):
    X, W, b, stride, padding, X_col = cache
    n_filter, d_filter, h_filter, w_filter = W.shape

    db = np.sum(dout, axis=(0, 2, 3))
    db = db.reshape(n_filter, -1)

    dout_reshaped = dout.transpose(1, 2, 3, 0).reshape(n_filter, -1)
    dW = dout_reshaped @ X_col.T
    dW = dW.reshape(W.shape)

    W_reshape = W.reshape(n_filter, -1)
    dX_col = W_reshape.T @ dout_reshaped
    dX = col2im_indices(dX_col, X.shape, h_filter, w_filter, padding=padding, stride=stride)

    return dX, dW, db

In [4]:
x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=3).reshape((3, 1))

conv_param = {'stride': 2, 'pad': 1}
out, _ = conv_forward(x, w, b, stride=2, padding=1)
correct_out = np.array([[[[-0.08759809, -0.10987781],
                           [-0.18387192, -0.2109216 ]],
                          [[ 0.21027089,  0.21661097],
                           [ 0.22847626,  0.23004637]],
                          [[ 0.50813986,  0.54309974],
                           [ 0.64082444,  0.67101435]]],
                         [[[-0.98053589, -1.03143541],
                           [-1.19128892, -1.24695841]],
                          [[ 0.69108355,  0.66880383],
                           [ 0.59480972,  0.56776003]],
                          [[ 2.36270298,  2.36904306],
                           [ 2.38090835,  2.38247847]]]])

# Compare your output to ours; difference should be around 2e-8
print('Testing conv_forward_naive')
print('difference: ', rel_error(out, correct_out))

Testing conv_forward_naive
difference:  2.21214765759e-08


In [5]:
np.random.seed(231)
x = np.random.randn(4, 3, 5, 5)
w = np.random.randn(2, 3, 3, 3)
b = np.random.randn(2,1)
dout = np.random.randn(4, 2, 5, 5)
conv_param = {'stride': 1, 'pad': 1}

dx_num = eval_numerical_gradient_array(lambda x: conv_forward(x, w, b, stride=1, padding=1)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: conv_forward(x, w, b, stride=1, padding=1)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: conv_forward(x, w, b, stride=1, padding=1)[0], b, dout)

out, cache = conv_forward(x, w, b,  stride=1, padding=1)
dx, dw, db = conv_backward(dout, cache)
#print(dx)

# Your errors should be around 1e-8'
#print('Testing conv_backward_naive function')
print('dx error: ', rel_error(dx, dx_num))
print('dw error: ', rel_error(dw, dw_num))
print('db error: ', rel_error(db, db_num))

dx error:  1.57427019706e-08
dw error:  1.79479957992e-10
db error:  2.58482087291e-11


In [6]:
def im2col(image, filter_size=(3, 3), padding=(0, 0), stride=(1, 1)):
    M, C, h, w, = image.shape
    filter_height = filter_size[0]
    filter_width = filter_size[1]
    padding_height = padding[0]
    padding_width = padding[1]
    stride_height = stride[0]
    stride_width = stride[1]
    x_padded = np.pad(image, ((0, 0),
                              (0, 0),
                              (padding_height, padding_height),
                              (padding_width, padding_width)),
                      mode='constant')
    h_new = int((h - filter_height + 2 * padding_height) / stride_height + 1)
    w_new = int((w - filter_width + 2 * padding_width) / stride_width + 1)

    out = np.zeros((filter_width * filter_height * C, M * h_new * w_new), dtype=image.dtype)

    itr = 0
    for i in range(h_new):
        for j in range(w_new):
            for m in range(M):
                start_i = stride_height * i
                end_i = stride_height * i + filter_width
                start_j = stride_width * j
                end_j = stride_width * j + filter_height
                out[:, itr] = x_padded[m, :, start_i:end_i, start_j:end_j].ravel()
                itr += 1
    return out


In [7]:
img = np.arange(36).reshape((2, 2, 3, 3))
img

array([[[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]]],


       [[[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]],

        [[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]]]])

In [8]:
im_old = im2col(img, filter_size=(3, 3))
im_old

array([[ 0, 18],
       [ 1, 19],
       [ 2, 20],
       [ 3, 21],
       [ 4, 22],
       [ 5, 23],
       [ 6, 24],
       [ 7, 25],
       [ 8, 26],
       [ 9, 27],
       [10, 28],
       [11, 29],
       [12, 30],
       [13, 31],
       [14, 32],
       [15, 33],
       [16, 34],
       [17, 35]])

In [9]:
%load_ext Cython

In [10]:
%%cython -a

cimport cython

import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cdef im2col_inner(np.float64_t[:, :, :, :] x_padded, np.float64_t[:, :] out, int h_new, int w_new, int C, int M, 
                 int filter_height, int filter_width, int stride_height, int stride_width):
    cdef int itr = 0
    
    cdef int start_i
    cdef int end_i
    cdef int start_j
    cdef int end_j
    
    cdef int i
    cdef int j
    cdef int m 
    
    cdef int k
    cdef int c
    cdef int p_h
    cdef int p_w
                
    for i in range(h_new):
        for j in range(w_new):
            for m in range(M):
                start_i = stride_height * i
                end_i = stride_height * i + filter_width
                start_j = stride_width * j
                end_j = stride_width * j + filter_height              
                
                k = 0
                for c in range(C):
                    for p_h in range(start_i, end_i):
                        for p_w in range(start_j, end_j):
                            out[k, itr] = x_padded[m, c, p_h, p_w]
                            k += 1
                itr += 1
                

@cython.boundscheck(False)
@cython.wraparound(False)
def im2col_cython(np.float64_t[:, :, :, :] image, 
                  int filter_height=3,
                  int filter_width=3, 
                  int padding_height=0, 
                  int padding_width=0, 
                  int stride_height=1,
                  int stride_width=1):
    
    cdef int M = image.shape[0]
    cdef int C = image.shape[1]
    cdef int h = image.shape[2]
    cdef int w = image.shape[3]
    
    cdef np.float64_t[:, :, :, :]  x_padded = np.pad(image, ((0, 0),
                              (0, 0),
                              (padding_height, padding_height),
                              (padding_width, padding_width)),
                      mode='constant')
    
    cdef int h_new = int((h - filter_height + 2 * padding_height) / stride_height + 1)
    cdef int w_new = int((w - filter_width + 2 * padding_width) / stride_width + 1)

    cdef int col_height = filter_width * filter_height * C
    cdef int col_width = M * h_new * w_new
    
    cdef np.float64_t[:, :] out = np.zeros((col_height, col_width))

    im2col_inner(x_padded, out, h_new, w_new, C, M, filter_height, filter_width, stride_height, stride_width)
    
    return out

In [11]:
import numpy as np

img = np.random.normal(scale=1.0, size=(2, 2, 3, 3))

im_new = im2col_cython(img, filter_height=5, filter_width=5)
im_old = im2col(img, filter_size=(5, 5))

print(rel_error(im_new, im_old))

large_img = np.random.normal(scale=1.0, size=(128, 64, 128, 128))

#%timeit im2col(large_img)
%timeit im2col_cython(large_img)

#rel_error(im2col(large_img), im2col_cython(large_img))

0.0
7.66 s ± 70.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
img

array([[[[ 1.22298233,  1.48723814, -0.6953817 ],
         [-0.76623039, -0.03463104,  0.1237851 ],
         [-0.47704517,  0.03203731, -0.67566443]],

        [[ 2.88187162,  0.54230072,  0.71058656],
         [-1.95394435,  0.59465664,  0.8706314 ],
         [-1.61928191,  0.15475624, -0.65885797]]],


       [[[ 1.08514262, -0.23300164, -0.52626964],
         [ 2.45727427,  0.19714757, -0.95348796],
         [-0.08733693,  0.29826634,  0.02558497]],

        [[-0.70464431,  0.31079506, -0.09970045],
         [-0.48568803,  0.66944594, -0.74343034],
         [ 0.38222283, -0.27093515, -0.21413126]]]])

In [13]:
import numpy
new_img = im2col_cython(img, filter_height=3, filter_width=3)
np.asarray(new_img)

array([[ 1.22298233,  1.08514262],
       [ 1.48723814, -0.23300164],
       [-0.6953817 , -0.52626964],
       [-0.76623039,  2.45727427],
       [-0.03463104,  0.19714757],
       [ 0.1237851 , -0.95348796],
       [-0.47704517, -0.08733693],
       [ 0.03203731,  0.29826634],
       [-0.67566443,  0.02558497],
       [ 2.88187162, -0.70464431],
       [ 0.54230072,  0.31079506],
       [ 0.71058656, -0.09970045],
       [-1.95394435, -0.48568803],
       [ 0.59465664,  0.66944594],
       [ 0.8706314 , -0.74343034],
       [-1.61928191,  0.38222283],
       [ 0.15475624, -0.27093515],
       [-0.65885797, -0.21413126]])

In [14]:
im_old = im2col(img, filter_size=(3, 3))
print(im_old)

[[ 1.22298233  1.08514262]
 [ 1.48723814 -0.23300164]
 [-0.6953817  -0.52626964]
 [-0.76623039  2.45727427]
 [-0.03463104  0.19714757]
 [ 0.1237851  -0.95348796]
 [-0.47704517 -0.08733693]
 [ 0.03203731  0.29826634]
 [-0.67566443  0.02558497]
 [ 2.88187162 -0.70464431]
 [ 0.54230072  0.31079506]
 [ 0.71058656 -0.09970045]
 [-1.95394435 -0.48568803]
 [ 0.59465664  0.66944594]
 [ 0.8706314  -0.74343034]
 [-1.61928191  0.38222283]
 [ 0.15475624 -0.27093515]
 [-0.65885797 -0.21413126]]


In [15]:
rel_error(im_old, new_img)

0.0

In [16]:
4780/395

12.10126582278481

In [17]:
%%cython -a

cimport cython

import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cdef col2img_inner(np.float64_t[:, :] cols, np.float64_t[:, :, :, :] x_padded, int filter_height, int filter_width,
                   int N, int C, int H, int W, int H_padded, int W_padded, int padding_height, int padding_width,
                   int stride_height, int stride_width):
    cdef int idx = 0
    cdef int i, j, m, c, sh, sw
    cdef int start_height, start_width, k
    cdef np.float64_t[:] col
    
    cdef int p = H_padded - filter_height + 1
    cdef int q = W_padded - filter_width + 1
    
    i =0
    while i < p:
    #for i in range(0, p, stride):
        j = 0
        while j < q:
        #for j in range(0, q, stride):
            for m in range(N):
                col = cols[:, idx]
                start_height = i
                start_width = j
                k = 0
                for c in range(C):
                    for sh in range(start_height, start_height + filter_height):
                        for sw in range(start_width, start_width + filter_width):
                            x_padded[m, c, sh, sw] += col[k]
                            k += 1
                idx += 1
            j += stride_width
        i += stride_height
    if padding_height > 0 or padding_width >0:
        return x_padded[:, :, padding_height:-padding_height, padding_width:-padding_width]
    else:
        return x_padded      

@cython.boundscheck(False)
@cython.wraparound(False)
def col2img_cython(np.float64_t[:, :] cols,  int batch_size, int no_color_challel, int image_height, 
                   int image_width, int filter_height=3, int filter_width=3, 
                   int padding_height=0, int padding_width=0, 
                   int stride_height=1, int stride_width=1):
    cdef int H_padded = image_height + 2 * padding_height
    cdef int W_padded = image_width + 2 * padding_width
    cdef np.float64_t[:, :, :, :]  x_padded = np.zeros((batch_size, no_color_challel, H_padded, W_padded))
    
    col2img_inner(cols, x_padded, filter_height, filter_width, batch_size, no_color_challel, 
                  image_height, image_width, H_padded, W_padded, padding_height, padding_width, 
                  stride_height, stride_width)
    return x_padded


In [18]:
img = np.random.normal(scale=1.0, size=(1, 2, 3, 3))
print(img)

[[[[-1.96221022  0.91386133 -1.65955002]
   [ 0.38563384 -2.56546445 -0.67339656]
   [-0.81926848 -1.33138613  0.17501128]]

  [[-0.45030696  1.14573192  3.21103599]
   [ 1.47017498 -0.89784141  0.44642588]
   [-0.70165903 -0.57639828  1.26679362]]]]


In [19]:
i2c = im2col(img, filter_size=(2, 2))
print(i2c)

[[-1.96221022  0.91386133  0.38563384 -2.56546445]
 [ 0.91386133 -1.65955002 -2.56546445 -0.67339656]
 [ 0.38563384 -2.56546445 -0.81926848 -1.33138613]
 [-2.56546445 -0.67339656 -1.33138613  0.17501128]
 [-0.45030696  1.14573192  1.47017498 -0.89784141]
 [ 1.14573192  3.21103599 -0.89784141  0.44642588]
 [ 1.47017498 -0.89784141 -0.70165903 -0.57639828]
 [-0.89784141  0.44642588 -0.57639828  1.26679362]]


In [20]:
N, C, H, W = img.shape
c2i_sliding = col2img_sliding(i2c, img.shape, filter_height=2, filter_width=2)
c2i_cyhton =  col2img_cython(i2c, batch_size=N, no_color_challel=C, image_height=H, image_width=W, filter_height=2, filter_width=2)
rel_error(c2i_sliding, c2i_cyhton)

0.0

In [21]:
N, C, H, W = img.shape
%timeit col2img_sliding(i2c, img.shape, filter_height=2, filter_width=2)
%timeit col2img_cython(i2c, batch_size=N, no_color_challel=C, image_height=H, image_width=W, filter_height=2, filter_width=2)

13.8 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.84 µs ± 5.51 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [22]:
c2i_sliding

array([[[[ -1.96221022,   1.82772265,  -1.65955002],
         [  0.77126769, -10.26185781,  -1.34679311],
         [ -0.81926848,  -2.66277226,   0.17501128]],

        [[ -0.45030696,   2.29146384,   3.21103599],
         [  2.94034997,  -3.59136562,   0.89285175],
         [ -0.70165903,  -1.15279656,   1.26679362]]]])

In [23]:
c2i_cyhton

<MemoryView of 'ndarray' at 0x7faca65751f0>

In [24]:
xx = np.zeros_like(img)
xx

array([[[[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]]]])

In [25]:
xx[0, 0, 0, 0] += -0.00278352
xx[0, 0, 0, 1] += -1.26840074
xx[0, 0, 1, 0] += 2.05517189
xx[0, 0, 1, 1] += -0.86800893

xx[0, 1, 0, 0] +=  1.48114097
xx[0, 1, 0, 1] += -0.92824366
xx[0, 1, 1, 0] += -0.56818538
xx[0, 1, 1, 1] += 0.81044009

xx[0, 0, 0, 1] += -1.2684007
xx[0, 0, 0, 2] += -0.72895969
xx[0, 0, 1, 1] += -0.86800893
xx[0, 0, 1, 2] += -1.35306576

xx

array([[[[-0.00278352, -2.53680144, -0.72895969],
         [ 2.05517189, -1.73601786, -1.35306576],
         [ 0.        ,  0.        ,  0.        ]],

        [[ 1.48114097, -0.92824366,  0.        ],
         [-0.56818538,  0.81044009,  0.        ],
         [ 0.        ,  0.        ,  0.        ]]]])

In [26]:
i2c

array([[-1.96221022,  0.91386133,  0.38563384, -2.56546445],
       [ 0.91386133, -1.65955002, -2.56546445, -0.67339656],
       [ 0.38563384, -2.56546445, -0.81926848, -1.33138613],
       [-2.56546445, -0.67339656, -1.33138613,  0.17501128],
       [-0.45030696,  1.14573192,  1.47017498, -0.89784141],
       [ 1.14573192,  3.21103599, -0.89784141,  0.44642588],
       [ 1.47017498, -0.89784141, -0.70165903, -0.57639828],
       [-0.89784141,  0.44642588, -0.57639828,  1.26679362]])

In [27]:
%timeit np.random.normal(scale=1.0, size=(128, 64, 128, 128))

4.4 s ± 137 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
x = np.random.normal(scale=1.0, size=(10, 10, 256, 256))

In [30]:
%timeit im2col_cython(x, 5, 5, 0, 0, 1, 1)

403 ms ± 6.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
